# Scratchwork: Cayley-Dickson Algebras

In [1]:
import finite_algebras as alg
import re
from cayley_dickson_utilities import *

The purpose of this notebook is to develop the functions required to perform the Cayley-Dickson construction within the ``finite_algebras`` module.

In the Python module, ``finite_algebras``, all elements are strings.

When direct products or Cayley-Dickson algebras are constructed, the component algebras' elements are concatentated (joined) using a string delimiter (':' by default).

So, operations like scalar multiplication, negation, conjugation, etc, on the newly constructed "compound" elements, require some string manipulation to accomplish.

According to [Shafer, 1966]

$a,b,c,d,\mu \in F$, $\mu \neq 0$, then $(a, b) \times (c, d) = (a c + \mu d \overline{b}, \overline{a} d + c b)$,

where $\overline{a} = a$ and $\overline{(a, b)} = (\overline{a}, -b)$

## Test Algebras

In [2]:
F3 = alg.generate_algebra_mod_n(3, elem_name='')
F3sqr = F3.sqr()
F3quad = F3sqr.sqr()

F4 = alg.generate_algebra_mod_n(4, elem_name='')
F4sqr = F4.sqr()

F5 = alg.generate_algebra_mod_n(5, elem_name='')
F5sqr = F5.sqr()

F7 = alg.generate_algebra_mod_n(7, elem_name='')
F7sqr = F7.sqr()

In [4]:
F3cda = F3.make_cayley_dickson_algebra()
F3cda.about()


** Group **
Name: F3_SQR
Instance ID: 4487421712
Description: Direct product of F3 with itself using complex multiplication
Order: 9
Identity: '0:0'
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0   '0:0'   '0:0'       1
      1   '0:1'   '0:2'       3
      2   '0:2'   '0:1'       3
      3   '1:0'   '2:0'       3
      4   '1:1'   '2:2'       3
      5   '1:2'   '2:1'       3
      6   '2:0'   '1:0'       3
      7   '2:1'   '1:2'       3
      8   '2:2'   '1:1'       3
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 0, 4, 5, 3, 7, 8, 6],
 [2, 0, 1, 5, 3, 4, 8, 6, 7],
 [3, 4, 5, 6, 7, 8, 0, 1, 2],
 [4, 5, 3, 7, 8, 6, 1, 2, 0],
 [5, 3, 4, 8, 6, 7, 2, 0, 1],
 [6, 7, 8, 0, 1, 2, 3, 4, 5],
 [7, 8, 6, 1, 2, 0, 4, 5, 3],
 [8, 6, 7, 2, 0, 1, 5, 3, 4]]


'<Group:F3_SQR, ID:4487421712>'

## Code Tests

In [ ]:
tests = ['2', '1:2', '2:1:0:2', '0:0']

In [ ]:
Fn = F3
FnSqr = F3sqr
FnQuad = F3quad

In [ ]:
scalar = '2'
for test in tests:
    print(f"{scalar} x {test} = {scalar_mult(scalar, test, Fn)}")

In [ ]:
for test in tests:
    print(f"-({test}) = {negate(test, Fn)}")

In [ ]:
for test in tests:
    print(f"conj({test}) = {conjugate(test, Fn)}")

In [ ]:
for test in tests[:2]: # just look at the first two items in tests
    print(f"|{test}|^2 = {norm(test, Fn, FnSqr)}")

In [ ]:
for test in tests[1:2]: # just look at the first two items in tests
    print(f"inv({test}) = {inverse(test, Fn, FnSqr)}")

In [ ]:
FnSqr.mult_inv(tests[1])  # The better way to obtain an inverse

In [ ]:
FnSqr.mult(tests[1], inverse(tests[1], Fn, FnSqr))

In [ ]:
try:
    inverse('0:0', Fn, FnSqr)
except Exception as exc:
    print(exc)

## String Manipulation

In [ ]:
x, y = split_element('a:b:c:d')
x, y

In [ ]:
u, v = split_element(x)
u, v

In [ ]:
split_element('a:b:c:d:e:f:g:h')

In [ ]:
split_element('a')

In [ ]:
split_element('')

## Cayley-Dickson Conjugates & Multiplication

In [ ]:
conj = conjugate

In [ ]:
print(conjugate('1', F7))
print(conjugate('1:2', F7))
print(conjugate('1:2:3:4', F7))

In [ ]:
x1 = '5:3'
y1 = '2:4'

In [ ]:
x1y1 = multiply(x1, y1, F7)
print(f"{x1} * {y1} = {x1y1}")
print(F7sqr.mult(x1, y1))

x1y1 == F7sqr.mult(x1, y1)

In [ ]:
All_OK = True
count = 0

for x1 in F7sqr.elements:
    for y1 in F7sqr.elements:
        count += 1
        xy_sqr = F7sqr.mult(x1, y1)
        xy_str = multiply(x1, y1, F7)
        if not xy_sqr == xy_str:
            print(x1, y1, xy_sqr, xy_str)
            All_OK = False

print(f"All {count} products OK? {All_OK}")

In [ ]:
x2 = '5'
y2 = '2'

In [ ]:
F7.elements

In [ ]:
x2y2 = multiply(x2, y2, F7)
print(f"{x2} * {y2} = {x2y2}")
print(F7.mult(x2, y2))

x2y2 == F7.mult(x2, y2)